# Análisis exploratorio de los datos

Los datasets a utilizar son los siguientes:
- `tf_000000000000.csv`: contiene las características musicales y metadata de las canciones a considerar.
- `.csv`: contiene información acerca de sesiones de usuarios de Spotify.